<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [9]</a>'.</span>

# 1. Parameters

In [1]:
cases_dir = 'cases/unset'
metadata_file = 'input/metadata-subsample-pangolin.tsv'
build_tree = False

In [2]:
# Parameters
cases_dir = "cases/case-20"
iterations = 3
number_samples = 20
build_tree = True


In [3]:
from pathlib import Path
import imp
fp, pathname, description = imp.find_module('gdi_benchmark', ['../../lib'])
gdi_benchmark = imp.load_module('gdi_benchmark', fp, pathname, description)

cases_dir_path = Path(cases_dir)

case_name = str(cases_dir_path.name)
index_path = cases_dir_path / 'index'

output_api_path = cases_dir_path / 'query-api.tsv'
output_cli_path = cases_dir_path / 'query-cli.tsv'

/tmp/ipykernel_52242/108103996.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


# 2. Benchmark command-line

In [4]:
import pandas as pd
import genomics_data_index.api as gdi

def benchmark_cli_index(name: str, index_path: Path, build_tree: bool) -> pd.DataFrame:
    benchmark_commands = {
        'query hasa':               f'gdi --project-dir {index_path} --ncores 1 query "hasa:hgvs_gn:NC_045512.2:S:p.D614G"',
        'query isa':                f'gdi --project-dir {index_path} --ncores 1 query "isa:Switzerland/100108/2020"',
        'query --summary':          f'gdi --project-dir {index_path} --ncores 1 query "hasa:hgvs_gn:NC_045512.2:S:p.D614G" --summary',
        'query --features-summary': f'gdi --project-dir {index_path} --ncores 1 query --features-summary mutations',
        'list samples':             f'gdi --project-dir {index_path} --ncores 1 list samples',
    }
    
    if build_tree:
        benchmark_commands['query isin'] = f'gdi --project-dir {index_path} --ncores 1 query --reference-name NC_045512 "isin_5_substitutions:Switzerland/100108/2020"'
    
    db = gdi.GenomicsDataIndex.connect(index_path)
    number_samples = db.count_samples()
    number_features_no_unknown = db.count_mutations(reference_genome='NC_045512', include_unknown=False)
    number_features_all = db.count_mutations(reference_genome='NC_045512', include_unknown=True)
    iterations = 10
    
    benchmarker = gdi_benchmark.QueryBenchmarkHandler()
    return benchmarker.benchmark_cli(name=name, kind_commands=benchmark_commands, number_samples=number_samples,
                                    number_features_no_unknown=number_features_no_unknown, number_features_all=number_features_all,
                                     iterations=iterations)

In [5]:
cli_df = benchmark_cli_index(name=case_name, index_path=index_path, build_tree=build_tree)
cli_df.head(3)

/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-manuscript/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f18b8ea4f40>
  self.pid = os.fork()


/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-manuscript/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f18b8ea4f40>
  self.pid = os.fork()


/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-manuscript/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f18b8ea4f40>
  self.pid = os.fork()


/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-manuscript/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f18b8ea4f40>
  self.pid = os.fork()


/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-manuscript/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f18b8ea4f40>
  self.pid = os.fork()


...

/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-manuscript/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f18b8ea4f40>
  self.pid = os.fork()


/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-manuscript/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f18b8ea4f40>
  self.pid = os.fork()


/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-manuscript/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f18b8ea4f40>
  self.pid = os.fork()


/home/CSCScience.ca/apetkau/miniconda3/envs/gdi-manuscript/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f18b8ea4f40>
  self.pid = os.fork()


,Name,Kind,Iteration,Number samples,Number features (no unknown),Number features (all),Runtime,Memory (max),Mmemory (max/process)
0,case-20,query hasa,1,20,247,8624,3.22,249303040.0,249303040.0
0,case-20,query hasa,2,20,247,8624,3.23,250118144.0,250118144.0
0,case-20,query hasa,3,20,247,8624,3.09,249978880.0,249978880.0


In [6]:
cli_df.to_csv(output_cli_path, sep='\t', index=False)

# 3. Test query API

## 3.1. Load (example) metadata

The simulated data is based off of real sample names and a real tree. So I can load up real metadata and attach it to a query (though the mutations and reference genome are all simulated).

In [7]:
import pandas as pd

metadata_df = pd.read_csv(metadata_file, sep='\t')
metadata_df.head(2)

,strain,virus,gisaid_epi_isl,genbank_accession,sra_accession,date,region,country,division,location,...,scorpio_conflict,scorpio_notes,version,pangolin_version,scorpio_version,constellation_version,is_designated,qc_status,qc_notes,note
0,Scotland/QEUH-3DF3D03/2022,ncov,EPI_ISL_12630193,OW855487,ERR9720965,2022-05-03,Europe,United Kingdom,Scotland,NaN,...,0.0,scorpio call: Alt alleles 64; Ref alleles 0; A...,PANGO-v1.14,4.1.2,0.3.17,v0.1.10,True,pass,Ambiguous_content:0.02,Assigned from designation hash.
1,Denmark/DCGC-384569/2022,ncov,EPI_ISL_10258944,OX022835,ERS12093125,2022-02-15,Europe,Denmark,Syddanmark,NaN,...,0.0,scorpio call: Alt alleles 64; Ref alleles 0; A...,PANGO-v1.14,4.1.2,0.3.17,v0.1.10,True,pass,Ambiguous_content:0.02,Assigned from designation hash.


## 3.2. Define benchmark cases

In [8]:
from typing import List
import genomics_data_index.api as gdi

def benchmark_api_index(name: str, index_path: Path, build_tree: bool) -> pd.DataFrame:
    db = gdi.GenomicsDataIndex.connect(index_path)
    q_no_join = db.samples_query(reference_name='NC_045512', universe='mutations')
    q_join = db.samples_query(reference_name='NC_045512', universe='mutations').join(metadata_df, sample_names_column='strain')

    q = q_join.hasa('hgvs_gn:NC_045512.2:S:p.D614G')
    r = q_join.hasa('hgvs_gn:NC_045512.2:N:p.R203K')
    
    number_samples = db.count_samples()
    number_features_no_unknown = db.count_mutations(reference_genome='NC_045512', include_unknown=False)
    number_features_all = db.count_mutations(reference_genome='NC_045512', include_unknown=True)
    repeat = 10

    benchmark_cases = {
        'db.samples_query':      lambda: db.samples_query(reference_name='NC_045512', universe='mutations'),
        'q.join':                lambda: q_no_join.join(metadata_df, sample_names_column='strain'),
        'q.features_summary':    lambda: q_join.features_summary(),
        'q.features_comparison': lambda: q_join.features_comparison(sample_categories='lineage', categories_kind='dataframe', kind='mutations', unit='proportion'),
        'q.hasa':                lambda: q_join.hasa("hgvs_gn:NC_045512.2:N:p.R203K"),
        'q.isa':                 lambda: q_join.isa("Switzerland/100112/2020"),
        'q AND r':               lambda: q & r,
        'q.toframe':             lambda: q_join.toframe(),
        'q.summary':             lambda: q_join.summary(),
    }
    
    if build_tree:
        benchmark_cases['q.isin (distance)'] = lambda: q_join.isin("Switzerland/100108/2020", kind='distance', distance=5, units='substitutions')
        benchmark_cases['q.isin (mrca)'] = lambda: q_join.isin(["Switzerland/100108/2020", "FR993751"], kind='mrca')

    benchmarker = gdi_benchmark.QueryBenchmarkHandler()
    return benchmarker.benchmark_api(name=name, kind_functions=benchmark_cases,
                                       number_samples=number_samples,
                                       number_features_no_unknown=number_features_no_unknown,
                                       number_features_all=number_features_all,
                                       repeat=repeat)

## 3.3. Benchmark reads index

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [9]:
api_df = benchmark_api_index(name=case_name, index_path=index_path, build_tree=build_tree)
api_df.head(5)

Attempting to set global database_path_translator=<genomics_data_index.storage.model.db.DatabasePathTranslator.DatabasePathTranslator object at 0x7f1ab46f0040> but it is already set


Exception: Invalid number of matching leaves for sample [Switzerland/100108/2020], leaves []

In [ ]:
api_df.to_csv(output_api_path, sep='\t', index=False)